In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable
import time
import collections
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
from torch import Tensor, tensor
import pandas as pd
import openml

#from aeon.regression.sklearn import RotationForestRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from preprocessing.stream_transforms import normalize_mean_std_traindata, normalize_streams, augment_time, add_basepoint_zero
from utils.utils import print_name, print_shape
from models import ResNet, NeuralEulerODE, RidgeCVModule, E2EResNet

np.set_printoptions(precision=3, threshold=5) # Print options

In [ ]:
from models import FittableModule, create_layer
from models import StagewiseRandFeatBoostRegression
    
N = 100
D = 50
p = 30
d = 4
bottleneck_dim = 70

gen = torch.Generator().manual_seed(42)
X = torch.randn(N, D, generator=gen)
y = torch.randn(N, d, generator=gen)
model = StagewiseRandFeatBoostRegression(
        gen,
        hidden_dim = D,
        bottleneck_dim = bottleneck_dim,
        out_dim = d,
        n_layers = 5,
        upscale = "dense",
    )
_, _ = model.fit(X, y)

# OpenML code

In [3]:
# Fetch the collection with ID 353
collection = openml.study.get_suite(353)
dataset_ids = collection.data
metadata_list = []

# Fetch and process each dataset
for i, dataset_id in enumerate(dataset_ids):
    dataset = openml.datasets.get_dataset(dataset_id)
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        target=dataset.default_target_attribute
    )
    X = np.array(X)
    y = np.array(y)[..., None]
    
    # Determine if the dataset has categorical features
    has_categorical = any(categorical_indicator)
    
    # Extract the required metadata
    metadata = {
        'dataset_id': dataset.id,
        'name': dataset.name,
        'n_obs': int(dataset.qualities['NumberOfInstances']),
        'n_features': int(dataset.qualities['NumberOfFeatures']),
        '%_unique_y': len(np.unique(y))/len(y),
        'n_unique_y': len(np.unique(y)),
        'has_categorical': has_categorical
    }
    
    metadata_list.append(metadata)
    print(f" {i+1}/{len(dataset_ids)} Processed dataset {dataset.id}: {dataset.name}")

# Create a DataFrame from the metadata list
df_metadata = pd.DataFrame(metadata_list).sort_values('%_unique_y', ascending=False).set_index("dataset_id")
df_metadata.sort_values('%_unique_y', ascending=True)

# Display the metadata DataFrame
df_metadata.loc[44962, "has_categorical"] = True
df_metadata

 1/35 Processed dataset 44956: abalone
 2/35 Processed dataset 44957: airfoil_self_noise
 3/35 Processed dataset 44958: auction_verification
 4/35 Processed dataset 44959: concrete_compressive_strength
 5/35 Processed dataset 44963: physiochemical_protein
 6/35 Processed dataset 44964: superconductivity
 7/35 Processed dataset 44965: geographical_origin_of_music
 8/35 Processed dataset 44966: solar_flare
 9/35 Processed dataset 44969: naval_propulsion_plant
 10/35 Processed dataset 44971: white_wine
 11/35 Processed dataset 44972: red_wine
 12/35 Processed dataset 44973: grid_stability
 13/35 Processed dataset 44974: video_transcoding
 14/35 Processed dataset 44975: wave_energy
 15/35 Processed dataset 44976: sarcos
 16/35 Processed dataset 44977: california_housing
 17/35 Processed dataset 44978: cpu_activity
 18/35 Processed dataset 44979: diamonds
 19/35 Processed dataset 44980: kin8nm
 20/35 Processed dataset 44981: pumadyn32nh
 21/35 Processed dataset 44983: miami_housing
 22/35 P

,name,n_obs,n_features,%_unique_y,n_unique_y,has_categorical
dataset_id,,,,,,
44973,grid_stability,10000,13,1.000000,10000,False
44975,wave_energy,72000,49,0.999903,71993,False
44980,kin8nm,8192,9,0.999878,8191,False
44981,pumadyn32nh,8192,33,0.999878,8191,False
45402,space_ga,3107,7,0.999356,3105,False
44958,auction_verification,2043,8,0.998042,2039,True
44994,cars,804,18,0.992537,798,False
44957,airfoil_self_noise,1503,6,0.968729,1456,False
44970,QSAR_fish_toxicity,908,7,0.910793,827,False


# Download single dataset

In [4]:
def load_openml_dataset(dataset_id, 
                        normalize_X:bool = True,
                        normalize_y:bool = True,
                        train_test_size:float = 0.7,
                        split_seed:int = 0,
                        device="cpu",
                        ) -> Tuple[np.ndarray, np.ndarray]:
    # Fetch dataset from OpenML by its ID
    dataset = openml.datasets.get_dataset(dataset_id)
    df, _, categorical_indicator, attribute_names = dataset.get_data()
    df.dropna(inplace=True)
    y = np.array(df.pop(dataset.default_target_attribute))[..., None]
    X = np.array(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_test_size, random_state=split_seed)

    #normalize
    if normalize_X:
        X_train, X_test = normalize_mean_std_traindata(X_train, X_test)
        X_train = np.clip(X_train, -3, 3)
        X_test = np.clip(X_test, -3, 3)
    if normalize_y:
        y_train, y_test = normalize_mean_std_traindata(y_train, y_test)

    return (torch.tensor(X_train.astype(np.float32), requires_grad=False, device=device),
            torch.tensor(X_test.astype(np.float32), requires_grad=False, device=device),
            torch.tensor(y_train.astype(np.float32), requires_grad=False, device=device),
            torch.tensor(y_test.astype(np.float32), requires_grad=False, device=device))

#dataset_id = 44971  # Replace with the dataset ID you want
dataset_id = 44971 #44970
X_train, X_test, y_train, y_test = load_openml_dataset(dataset_id, False, False)


# Plot Activations

In [5]:
# import torch
# import torch.nn as nn
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# from plotly.subplots import make_subplots

# def get_activation(name, activations):
#     def hook(model, input, output):
#         activations[name] = output.detach()
#     return hook


# def register_hooks(model, activations):
#     for name, layer in model.named_modules():
#         print(name)
#         if ".dense" not in name:
#             layer.register_forward_hook(get_activation(name, activations))



# def neuron_distribution_for_each_layer(X_train, y_train, X_test):
#     D = X_train.shape[1]
#     n_layers = 2
#     g1 = torch.Generator().manual_seed(0)
#     model = SampledEulerODE(g1, D, 10*D, n_layers, upsample_module='sampled', sampling_method='gradient')
#     #model = SampledResNet(g1, D, 10*D, 10*D, n_layers, upsample_module='sampled', sampling_method='gradient')
#     model.fit(X_train, y_train)

#     activations = {}
#     register_hooks(model, activations)
    
#     # Forward pass
#     model(X_test)
    
#     # Plot input data distribution
#     fig = make_subplots(rows=1, cols=1)
#     fig.add_trace(go.Histogram(x=X_train.flatten().cpu().numpy(), nbinsx=50, name='Train', histnorm='probability density', opacity=0.5))
#     fig.add_trace(go.Histogram(x=X_test.flatten().cpu().numpy(), nbinsx=50, name='Test', histnorm='probability density', opacity=0.5))
#     fig.update_layout(title_text='Input Data Distribution', xaxis_title='Input Feature Value', yaxis_title='Probability Density', barmode='overlay')
#     fig.show()

#     # Plot activations
#     for name, activation in activations.items():
#         fig = make_subplots(rows=1, cols=1)
#         fig.add_trace(go.Histogram(x=activation.flatten().cpu().numpy(), nbinsx=50, name='Activation', histnorm='probability density', opacity=0.5))
#         fig.update_layout(title_text=f'Activations at Layer: {name}', xaxis_title='Activation Value', yaxis_title='Probability Density', barmode='overlay')
#         fig.show()


# neuron_distribution_for_each_layer(X_train, y_train, X_test)

# Fit on a dataset

In [6]:
def run_allmodels_1dataset(
        generator: torch.Generator,
        X_train: Tensor,
        y_train: Tensor,
        X_test: Tensor,
        y_test: Tensor,
        ):
    
    D = X_train.shape[1]
    hidden_size = 512
    bottleneck_dim = hidden_size

    # (name, model, kwargs). kwargs separate to save memory
    model_list = [
        ["RidgeCV", RidgeCVModule, {}],

        ["T=3 End2End", E2EResNet,
                {"generator": generator,
                "in_dim": D,
                "hidden_size": hidden_size,
                "bottleneck_dim": bottleneck_dim,
                "out_dim": 1,
                "n_blocks": 2,
                "activation": nn.Tanh(),
                "loss": nn.MSELoss(),
                "lr": 1e-3,
                "epochs": 50,
                "batch_size": 64,}
                ],

        ["T=1 Dense", ResNet,
                {"generator": generator,
                 "in_dim": D,
                 "hidden_size": hidden_size,
                 "bottleneck_dim": None,
                 "n_blocks": 0,
                 "upsample_layer": "dense",}
                 ],

        ["T=1 SWIM Grad", ResNet,
                {"generator": generator,
                "in_dim": D,
                "hidden_size": hidden_size,
                "bottleneck_dim": None,
                "n_blocks": 0,
                "upsample_layer": "SWIM",}
                ],
        
        ["T=1 SWIM Unif", ResNet,
                {"generator": generator,
                "in_dim": D,
                "hidden_size": hidden_size,
                "bottleneck_dim": None,
                "n_blocks": 0,
                "upsample_layer": "SWIM",
                "sampling_method": "uniform",}
                ],
    ]

    for n_blocks in [2, 4]:
        model_list += [
        [f"T={n_blocks+1} ResSWIM Grad-dense", ResNet,
                {"generator": generator,
                "in_dim": D,
                "hidden_size": hidden_size,
                "bottleneck_dim": bottleneck_dim,
                "n_blocks": n_blocks,
                "upsample_layer": "SWIM",
                "res_layer1": "SWIM",
                "res_layer2": "dense",}
                ],

        [f"T={n_blocks+1} ResSWIM Grad-id", ResNet,
                {"generator": generator,
                "in_dim": D,
                "hidden_size": hidden_size,
                "bottleneck_dim": hidden_size,
                "n_blocks": n_blocks,
                "upsample_layer": "SWIM",
                "res_layer1": "SWIM",
                "res_layer2": "identity",}
                ],
                
        [f"T={n_blocks+1} ResDense", ResNet,
                {"generator": generator,
                "in_dim": D,
                "hidden_size": hidden_size,
                "bottleneck_dim": hidden_size,
                "n_blocks": n_blocks,
                "upsample_layer": "dense",
                "res_layer1": "dense",
                "res_layer2": "identity",}
                ],
        ]
        
    for n_layers in range(0, 50, 5):
        model_list += [
        [f"StagewiseRandFeatBoost_{n_layers}", StagewiseRandFeatBoostRegression,
                {"generator": generator,
                "hidden_dim": hidden_size,
                "bottleneck_dim": bottleneck_dim,
                "out_dim": 1,
                "n_layers": n_layers,
                "activation": nn.Tanh(),
                "l2_reg": 1,  #TODO experiment with much higher l2reg than 0.01
                "feature_type": "SWIM",
                "boost_lr": 1.0,
                "upscale": "dense",}
                ],
        ]
    
    results = []
    model_names = []
    for name, model, model_args in model_list:
        t0 = time.perf_counter()
        model = model(**model_args).to(X_train.device)
        pred_train, _ = model.fit(X_train, y_train)
        t1 = time.perf_counter()
        pred_test = model(X_test)
        t2 = time.perf_counter()
        rmse_train = root_mean_squared_error(y_train.cpu(), pred_train.cpu().detach()) 
        rmse_test = root_mean_squared_error(y_test.cpu(), pred_test.cpu().detach())

        result = np.array( [rmse_train, rmse_test, t1-t0, t2-t1] )
        results.append( result )
        model_names.append( name )

    return model_names, results



def run_all_experiments(
        dataset_ids: List,
        name_save: str = "PLACEHOLDER",
        device="cuda",
        ):
    # Fetch and process each dataset
    experiments = {}
    for i, dataset_id in enumerate(dataset_ids):
        X_train, X_test, y_train, y_test = load_openml_dataset(dataset_id, split_seed=0, device=device)
        generator = torch.Generator(device=device).manual_seed(999)
        results = run_allmodels_1dataset(
            generator, X_train, y_train, X_test, y_test, 
            )
        experiments[dataset_id] = results
        print(f" {i+1}/{len(dataset_ids)} Processed dataset {dataset_id}")

    # Save results
    # Assuming experiments is a dict where keys are dataset names and values are tuples (model_names, results)
    attributes = ["RMSE_train", "RMSE_test", "t_fit", "t_feat"]
    data_list = []
    # Process the data
    for dataset_name, (model_names, results) in experiments.items():
        dataset_data = {}
        for attr_idx, attribute in enumerate(attributes):
            for model_idx, model_name in enumerate(model_names):
                dataset_data[(attribute, model_name)] = results[model_idx][attr_idx]
        data_list.append(pd.DataFrame(dataset_data, index=[dataset_name]))

    # Combine all datasets into a single DataFrame
    df = pd.concat(data_list)
    df = df.sort_index(axis=1)
    print(df)
    df.to_pickle(f"OpenML_reg_{name_save}.pkl")
    return df

In [7]:
dataset_ids_not_categorical = list(df_metadata.query("has_categorical == False").index.values)
dataset_ids_not_categorical = sorted([int(x) for x in dataset_ids_not_categorical])
run_all_experiments(dataset_ids_not_categorical, name_save="FIRSTBOOST512lambda1")

100%|██████████| 50/50 [00:03<00:00, 15.14it/s]


 1/20 Processed dataset 44957


100%|██████████| 50/50 [00:02<00:00, 21.85it/s]


 2/20 Processed dataset 44959


100%|██████████| 50/50 [00:01<00:00, 26.33it/s]


...

100%|██████████| 50/50 [00:01<00:00, 26.07it/s]


 19/20 Processed dataset 44994


100%|██████████| 50/50 [00:06<00:00,  7.92it/s]


 20/20 Processed dataset 45402
      RMSE_test                                                     \
        RidgeCV StagewiseRandFeatBoost_0 StagewiseRandFeatBoost_10   
44957  0.674484                 0.674491                  0.326598   
44959  0.542088                 0.542225                  0.356812   
44960  0.304327                 0.304471                  0.192110   
44963  0.839762                 0.839764                  0.720746   
44964  0.517322                 0.517305                  0.411687   
44965  0.914663                 0.916203                  0.909711   
44969  0.413739                 0.399114                  0.026289   
44970  0.666021                 0.667432                  0.741133   
44971  0.878211                 0.878129                  0.856063   
44972  0.766536                 0.766392                  0.845825   
44973  0.595158                 0.595165                  0.250120   
44975  0.006491                 0.006491                  0

RMSE_test                                                     \
        RidgeCV StagewiseRandFeatBoost_0 StagewiseRandFeatBoost_10   
44957  0.674484                 0.674491                  0.326598   
44959  0.542088                 0.542225                  0.356812   
44960  0.304327                 0.304471                  0.192110   
44963  0.839762                 0.839764                  0.720746   
44964  0.517322                 0.517305                  0.411687   
44965  0.914663                 0.916203                  0.909711   
44969  0.413739                 0.399114                  0.026289   
44970  0.666021                 0.667432                  0.741133   
44971  0.878211                 0.878129                  0.856063   
44972  0.766536                 0.766392                  0.845825   
44973  0.595158                 0.595165                  0.250120   
44975  0.006491                 0.006491                  0.006485   
44976  0.294862                 0.294862                  0.185641   
44977  0.587720                 0.587715                  0.499144   
44978  0.323560                 0.323675                  0.149001   
44980  0.771311                 0.771336                  0.445765   
44981  0.904478                 0.904057                  0.904056   
44983  0.586507                 0.586497                  0.345728   
44994  0.296725                 0.296724                  0.295729   
45402  0.706690                 0.706860                  0.570821   

                                                           \
      StagewiseRandFeatBoost_15 StagewiseRandFeatBoost_20   
44957                  0.328366                  0.314254   
44959                  0.378886                  0.325914   
44960                  0.205260                  0.187507   
44963                  0.717604                  0.717611   
44964                  0.407239                  0.407266   
44965                  0.907255                  0.907011   
44969                  0.038981                  0.049000   
44970                  0.679183                  0.617449   
44971                  0.855164                  0.895499   
44972                  0.876527                  0.844666   
44973                  0.236416                  0.235026   
44975                  0.006483                  0.006490   
44976                  0.184711                  0.183739   
44977                  0.515810                  0.499861   
44978                  0.143936                  0.141868   
44980                  0.368507                  0.348240   
44981                  0.904055                  0.904055   
44983                  0.336431                  0.338367   
44994                  0.227546                  0.295319   
45402                  0.564003                  0.563967   

                                                           \
      StagewiseRandFeatBoost_25 StagewiseRandFeatBoost_30   
44957                  0.343630                  0.351402   
44959                  0.335755                  0.453864   
44960                  0.239054                  0.179068   
44963                  0.717060                  0.720186   
44964                  0.396595                  0.407522   
44965                  0.904485                  0.902259   
44969                  0.019856                  0.018542   
44970                  0.643343                  0.810779   
44971                  0.897548                  0.992478   
44972                  0.873176                  0.998274   
44973                  0.303932                  0.234628   
44975                  0.006465                  0.006475   
44976                  0.178020                  0.172097   
44977                  0.506922                  0.527786   
44978                  0.142302                  0.141127   
44980                  0.347302                  0.341979   
44981                  0

In [8]:
df_reg = pd.read_pickle("OpenML_reg_FIRSTBOOST128.pkl")
df_reg["RMSE_test"].mean().sort_values()

T=3 End2End                  0.445475
StagewiseRandFeatBoost_45    0.462458
StagewiseRandFeatBoost_20    0.464753
StagewiseRandFeatBoost_15    0.465226
StagewiseRandFeatBoost_30    0.465412
StagewiseRandFeatBoost_25    0.466015
StagewiseRandFeatBoost_35    0.467795
StagewiseRandFeatBoost_10    0.475439
StagewiseRandFeatBoost_5     0.475702
StagewiseRandFeatBoost_40    0.485924
T=3 ResSWIM Grad-id          0.486957
T=3 ResSWIM Grad-dense       0.487220
T=5 ResSWIM Grad-dense       0.487323
T=5 ResSWIM Grad-id          0.489470
T=1 Dense                    0.497886
T=1 SWIM Grad                0.498980
T=3 ResDense                 0.511270
T=1 SWIM Unif                0.522667
T=5 ResDense                 0.525264
StagewiseRandFeatBoost_0     0.578799
RidgeCV                      0.579533
dtype: float64

In [9]:
df_reg["RMSE_test"].rank(axis=1).mean().sort_values()

StagewiseRandFeatBoost_45     6.85
StagewiseRandFeatBoost_20     7.45
StagewiseRandFeatBoost_35     7.95
StagewiseRandFeatBoost_15     8.30
StagewiseRandFeatBoost_30     8.35
StagewiseRandFeatBoost_40     8.65
StagewiseRandFeatBoost_25     8.70
T=3 End2End                   9.45
StagewiseRandFeatBoost_5      9.75
StagewiseRandFeatBoost_10    10.10
T=1 Dense                    10.95
T=3 ResSWIM Grad-id          11.00
T=5 ResSWIM Grad-dense       11.30
T=3 ResSWIM Grad-dense       11.30
T=5 ResSWIM Grad-id          11.75
T=1 SWIM Grad                13.00
T=3 ResDense                 13.25
T=5 ResDense                 13.80
T=1 SWIM Unif                14.50
StagewiseRandFeatBoost_0     17.30
RidgeCV                      17.30
dtype: float64

In [10]:
df_reg["RMSE_train"].mean().sort_values()

T=3 End2End                  0.295489
StagewiseRandFeatBoost_45    0.340766
StagewiseRandFeatBoost_35    0.347595
StagewiseRandFeatBoost_40    0.348153
StagewiseRandFeatBoost_30    0.357114
StagewiseRandFeatBoost_25    0.363361
StagewiseRandFeatBoost_20    0.371707
StagewiseRandFeatBoost_15    0.383592
StagewiseRandFeatBoost_10    0.414180
StagewiseRandFeatBoost_5     0.434197
T=3 ResSWIM Grad-dense       0.457521
T=5 ResSWIM Grad-dense       0.458178
T=5 ResSWIM Grad-id          0.459707
T=3 ResSWIM Grad-id          0.460371
T=1 Dense                    0.471736
T=1 SWIM Grad                0.476849
T=3 ResDense                 0.480694
T=5 ResDense                 0.489757
T=1 SWIM Unif                0.499858
StagewiseRandFeatBoost_0     0.568045
RidgeCV                      0.569632
dtype: float64

In [11]:
df_reg["RMSE_train"].rank(axis=1).mean().sort_values()

StagewiseRandFeatBoost_45     2.450
StagewiseRandFeatBoost_40     3.300
StagewiseRandFeatBoost_35     3.500
StagewiseRandFeatBoost_30     4.450
StagewiseRandFeatBoost_25     5.900
T=3 End2End                   6.300
StagewiseRandFeatBoost_20     6.550
StagewiseRandFeatBoost_15     7.000
StagewiseRandFeatBoost_10     9.200
StagewiseRandFeatBoost_5     11.350
T=5 ResSWIM Grad-dense       13.450
T=3 ResSWIM Grad-dense       13.550
T=1 Dense                    13.600
T=3 ResSWIM Grad-id          14.200
T=5 ResSWIM Grad-id          14.600
T=5 ResDense                 14.650
T=3 ResDense                 14.800
T=1 SWIM Grad                15.950
T=1 SWIM Unif                17.050
StagewiseRandFeatBoost_0     19.025
RidgeCV                      20.125
dtype: float64

In [12]:
df_reg = pd.read_pickle("OpenML_reg_FIRSTBOOST512.pkl")    # BAD TEST PERFORMANCE DUE TO REGULARIZATION ??
df_reg["RMSE_test"].mean().sort_values()

T=5 ResSWIM Grad-dense       0.429159
T=5 ResSWIM Grad-id          0.432472
T=3 ResSWIM Grad-id          0.436384
T=3 ResSWIM Grad-dense       0.438637
T=3 ResDense                 0.443628
T=1 Dense                    0.449241
T=1 SWIM Grad                0.450871
T=5 ResDense                 0.452306
T=3 End2End                  0.462525
T=1 SWIM Unif                0.465533
StagewiseRandFeatBoost_5     0.499303
StagewiseRandFeatBoost_15    0.516132
StagewiseRandFeatBoost_10    0.520637
StagewiseRandFeatBoost_30    0.531521
StagewiseRandFeatBoost_20    0.535844
StagewiseRandFeatBoost_45    0.540014
StagewiseRandFeatBoost_40    0.548790
StagewiseRandFeatBoost_35    0.554466
StagewiseRandFeatBoost_25    0.570587
StagewiseRandFeatBoost_0     0.578945
RidgeCV                      0.579533
dtype: float64

In [13]:
df_reg["RMSE_test"].rank(axis=1).mean().sort_values()

T=5 ResSWIM Grad-dense        8.50
StagewiseRandFeatBoost_45     8.85
T=5 ResSWIM Grad-id           9.20
StagewiseRandFeatBoost_35     9.25
T=3 ResDense                  9.30
StagewiseRandFeatBoost_30     9.40
T=3 ResSWIM Grad-dense        9.40
T=3 ResSWIM Grad-id           9.80
StagewiseRandFeatBoost_40     9.95
T=3 End2End                  10.20
T=1 Dense                    10.25
StagewiseRandFeatBoost_25    10.45
T=5 ResDense                 10.90
StagewiseRandFeatBoost_20    11.05
StagewiseRandFeatBoost_15    11.20
T=1 SWIM Grad                11.35
StagewiseRandFeatBoost_10    11.60
StagewiseRandFeatBoost_5     12.55
T=1 SWIM Unif                13.45
RidgeCV                      17.05
StagewiseRandFeatBoost_0     17.30
dtype: float64

In [14]:
df_reg["RMSE_train"].mean().sort_values()

StagewiseRandFeatBoost_35    0.274765
StagewiseRandFeatBoost_30    0.285083
StagewiseRandFeatBoost_45    0.285548
StagewiseRandFeatBoost_40    0.287685
StagewiseRandFeatBoost_20    0.300381
StagewiseRandFeatBoost_25    0.301658
StagewiseRandFeatBoost_10    0.324424
StagewiseRandFeatBoost_15    0.327888
T=3 End2End                  0.334721
StagewiseRandFeatBoost_5     0.371574
T=5 ResSWIM Grad-dense       0.392778
T=5 ResSWIM Grad-id          0.394202
T=3 ResDense                 0.397092
T=3 ResSWIM Grad-id          0.399135
T=1 Dense                    0.400481
T=3 ResSWIM Grad-dense       0.402919
T=5 ResDense                 0.403114
T=1 SWIM Grad                0.420298
T=1 SWIM Unif                0.436566
StagewiseRandFeatBoost_0     0.567798
RidgeCV                      0.569632
dtype: float64

In [15]:
df_reg["RMSE_train"].rank(axis=1).mean().sort_values()

StagewiseRandFeatBoost_40     3.550
StagewiseRandFeatBoost_35     3.700
StagewiseRandFeatBoost_30     4.000
StagewiseRandFeatBoost_45     4.050
StagewiseRandFeatBoost_25     5.850
StagewiseRandFeatBoost_20     5.950
StagewiseRandFeatBoost_15     7.900
StagewiseRandFeatBoost_10     9.300
T=3 End2End                   9.650
StagewiseRandFeatBoost_5     11.650
T=3 ResDense                 12.350
T=5 ResDense                 12.800
T=1 Dense                    12.850
T=5 ResSWIM Grad-dense       12.900
T=5 ResSWIM Grad-id          13.350
T=3 ResSWIM Grad-dense       14.100
T=3 ResSWIM Grad-id          14.150
T=1 SWIM Grad                15.950
T=1 SWIM Unif                17.200
StagewiseRandFeatBoost_0     19.575
RidgeCV                      20.175
dtype: float64

In [16]:
df_reg = pd.read_pickle("OpenML_reg_FIRSTBOOST512lambda1.pkl")
df_reg["RMSE_test"].mean().sort_values()

T=5 ResSWIM Grad-dense       0.429140
T=5 ResSWIM Grad-id          0.432475
T=3 ResSWIM Grad-id          0.436382
T=3 ResSWIM Grad-dense       0.438584
StagewiseRandFeatBoost_20    0.439155
T=3 ResDense                 0.443628
StagewiseRandFeatBoost_35    0.443638
StagewiseRandFeatBoost_15    0.444118
StagewiseRandFeatBoost_25    0.445988
StagewiseRandFeatBoost_45    0.448495
T=1 Dense                    0.449241
T=1 SWIM Grad                0.450873
StagewiseRandFeatBoost_10    0.451973
T=5 ResDense                 0.452306
StagewiseRandFeatBoost_40    0.453608
T=3 End2End                  0.462525
StagewiseRandFeatBoost_5     0.462983
T=1 SWIM Unif                0.465533
StagewiseRandFeatBoost_30    0.468853
StagewiseRandFeatBoost_0     0.578945
RidgeCV                      0.579533
dtype: float64

In [17]:
df_reg["RMSE_test"].rank(axis=1).mean().sort_values()

StagewiseRandFeatBoost_35     7.20
StagewiseRandFeatBoost_45     7.75
StagewiseRandFeatBoost_40     7.85
StagewiseRandFeatBoost_20     8.90
T=5 ResSWIM Grad-dense        8.95
T=3 ResDense                  9.35
T=5 ResSWIM Grad-id           9.45
T=3 ResSWIM Grad-dense        9.50
T=3 ResSWIM Grad-id           9.95
StagewiseRandFeatBoost_25    10.05
T=1 Dense                    10.45
StagewiseRandFeatBoost_15    10.60
StagewiseRandFeatBoost_30    10.90
T=5 ResDense                 11.15
T=3 End2End                  11.60
StagewiseRandFeatBoost_10    11.65
T=1 SWIM Grad                11.70
StagewiseRandFeatBoost_5     13.70
T=1 SWIM Unif                13.75
RidgeCV                      18.15
StagewiseRandFeatBoost_0     18.40
dtype: float64

In [18]:
df_reg["RMSE_train"].mean().sort_values()

T=3 End2End                  0.334721
StagewiseRandFeatBoost_45    0.350544
StagewiseRandFeatBoost_40    0.358136
StagewiseRandFeatBoost_30    0.359853
StagewiseRandFeatBoost_25    0.363275
StagewiseRandFeatBoost_35    0.363362
StagewiseRandFeatBoost_15    0.372795
StagewiseRandFeatBoost_20    0.376107
StagewiseRandFeatBoost_10    0.387099
T=5 ResSWIM Grad-dense       0.392842
T=5 ResSWIM Grad-id          0.394172
T=3 ResDense                 0.397092
T=3 ResSWIM Grad-id          0.399132
T=1 Dense                    0.400481
T=3 ResSWIM Grad-dense       0.402892
T=5 ResDense                 0.403114
StagewiseRandFeatBoost_5     0.414447
T=1 SWIM Grad                0.420300
T=1 SWIM Unif                0.436566
StagewiseRandFeatBoost_0     0.567798
RidgeCV                      0.569632
dtype: float64

In [19]:
df_reg["RMSE_train"].rank(axis=1).mean().sort_values()

StagewiseRandFeatBoost_45     4.900
StagewiseRandFeatBoost_40     5.300
StagewiseRandFeatBoost_35     5.850
StagewiseRandFeatBoost_25     6.650
StagewiseRandFeatBoost_30     6.650
StagewiseRandFeatBoost_20     8.500
StagewiseRandFeatBoost_15     9.150
T=3 End2End                   9.150
T=3 ResDense                 10.200
T=5 ResDense                 10.450
T=1 Dense                    10.850
T=5 ResSWIM Grad-dense       10.950
StagewiseRandFeatBoost_10    11.200
T=5 ResSWIM Grad-id          11.600
T=3 ResSWIM Grad-dense       12.200
T=3 ResSWIM Grad-id          12.250
StagewiseRandFeatBoost_5     13.850
T=1 SWIM Grad                15.000
T=1 SWIM Unif                16.550
StagewiseRandFeatBoost_0     19.575
RidgeCV                      20.175
dtype: float64